## 📦 Step 1: Install Dependencies

In [ ]:
# Mount Google Drive (untuk save model & results)
from google.colab import drive
drive.mount('/content/drive')

# Check GPU
import torch
print(f"🖥️ GPU Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"   GPU: {torch.cuda.get_device_name(0)}")
    !nvidia-smi
else:
    print("⚠️ WARNING: GPU not detected! Training akan sangat lambat.")
    print("   Go to: Runtime → Change runtime type → Hardware accelerator → GPU")

In [ ]:
!pip install -q roboflow ultralytics onnx onnxruntime tqdm

print("✅ All dependencies installed!")

## 📥 Step 2: Download Dataset dari Roboflow
### ✅ Dataset sudah digabung di Roboflow (740 + 137 images, NO augmentation!)

In [ ]:
from roboflow import Roboflow

# Download MERGED dataset (kosan-hendra + detection-chess FIXED)
# Version 3 = Merged dataset TANPA augmentasi (original images only!)
print("📥 Downloading MERGED Dataset (v3 - NO AUGMENTATION)...")
print("   Dataset 1: kosan-hendra (~740 images ori)")
print("   Dataset 2: detection-chess FIXED (~137 images, 12 classes)")
print("   Total: ~877 images (ORIGINAL, tanpa augmentasi!)")
print()

rf = Roboflow(api_key="9ip0woBWDJ4vucWT4GfN")
project = rf.workspace("detection-chess").project("chess-detection-76mfe")
version = project.version(3)  # ← Version 3: Merged, NO augmentation!
dataset = version.download("yolov8")

print(f"\n✅ Dataset downloaded: {dataset.location}")
print(f"\n📊 Dataset Info:")
print(f"   Version: 3 (Merged + NO Augmentation)")
print(f"   Expected: ~877 original images")
print(f"   Classes: 12 (white-king, ..., black-pawn)")

print("\n" + "="*60)
print("✅ Dataset ready for training!")
print("="*60)

## ✅ Step 3: Verify Dataset (SKIP MERGE - Sudah Merged di Roboflow!)
### Dataset sudah digabung di Roboflow, tidak perlu merge manual!

In [ ]:
import os
import yaml

# ✅ SKIP MERGE - Dataset sudah digabung di Roboflow!
# Langsung gunakan dataset yang sudah di-download

print("="*60)
print("✅ DATASET VERIFICATION")
print("="*60)

# Verify dataset structure
dataset_path = dataset.location

print(f"\n📁 Dataset location: {dataset_path}")

# Count images
train_images = len([f for f in os.listdir(f"{dataset_path}/train/images") 
                     if f.endswith(('.jpg', '.png', '.jpeg'))])
train_labels = len([f for f in os.listdir(f"{dataset_path}/train/labels") 
                     if f.endswith('.txt')])

valid_images = len([f for f in os.listdir(f"{dataset_path}/valid/images") 
                     if f.endswith(('.jpg', '.png', '.jpeg'))])
valid_labels = len([f for f in os.listdir(f"{dataset_path}/valid/labels") 
                     if f.endswith('.txt')])

print(f"\n📊 Dataset Statistics:")
print(f"   Train: {train_images} images, {train_labels} labels")
print(f"   Valid: {valid_images} images, {valid_labels} labels")
print(f"   TOTAL: {train_images + valid_images} images")

# Verify classes
with open(f"{dataset_path}/data.yaml", 'r') as f:
    data_config = yaml.safe_load(f)

print(f"\n📋 Classes ({data_config['nc']}):")
for i, class_name in enumerate(data_config['names']):
    print(f"   {i}: {class_name}")

print("\n✅ Dataset verified - ready for training!")
print("="*60)

## 📋 Step 4: Prepare data.yaml for Training

In [ ]:
import yaml
import os

# Dataset path
dataset_yaml_path = f"{dataset.location}/data.yaml"

print("="*60)
print("📋 PREPARING DATA.YAML")
print("="*60)

# Read data.yaml
with open(dataset_yaml_path, 'r') as f:
    data_config = yaml.safe_load(f)

print(f"\n📄 Dataset Configuration:")
print(f"   Path: {data_config.get('path', dataset.location)}")
print(f"   Train: {data_config.get('train', 'train/images')}")
print(f"   Val: {data_config.get('val', 'valid/images')}")
print(f"   Classes: {data_config['nc']}")
print(f"\n   Class Names:")
for i, name in enumerate(data_config['names']):
    print(f"      {i}: {name}")

# Count final images
train_count = len([f for f in os.listdir(f"{dataset.location}/train/images") 
                    if f.endswith(('.jpg', '.png', '.jpeg'))])
val_count = len([f for f in os.listdir(f"{dataset.location}/valid/images") 
                  if f.endswith(('.jpg', '.png', '.jpeg'))])

print(f"\n📊 Final Image Counts:")
print(f"   Training: {train_count} images")
print(f"   Validation: {val_count} images")
print(f"   TOTAL: {train_count + val_count} images")

# Decision helper
print("\n" + "="*60)
print("💡 TRAINING RECOMMENDATION:")
print("="*60)

total = train_count + val_count

if total < 500:
    print("⚠️ Dataset kecil (<500 images)")
    print("➡️ Consider adding more data")
elif total < 1000:
    print("✅ Dataset size GOOD (~500-1000 images)")
    print("➡️ Built-in YOLO augmentation akan sangat membantu!")
else:
    print("🎉 Dataset size EXCELLENT (>1000 images)")
    print("➡️ Ready for high-quality training!")

print(f"\n📈 With YOLO built-in augmentation:")
print(f"   Original: {total} images")
print(f"   Augmented: ~{total * 10} variations per epoch (dynamic!)")
print(f"\n💡 Augmentation settings:")
print(f"   ✅ HSV (brightness, saturation, hue)")
print(f"   ✅ Mosaic (4-image combinations)")
print(f"   ✅ MixUp (image blending)")
print(f"   ✅ Geometric (rotation, translation, scale)")

print("\n✅ Data.yaml ready for training!")
print("="*60)

## ⚠️ SKIP: Manual Augmentation (TIDAK PERLU!)
### ✅ YOLO built-in augmentation sudah cukup!
### 🔽 Skip 2 cells di bawah ini - langsung ke Step 5 (Training)

In [ ]:
import albumentations as A
import cv2
import glob
import shutil
from tqdm import tqdm

# ⚠️ SKIP CELL INI! Manual augmentation tidak diperlukan!
# YOLO punya built-in augmentation yang lebih baik (hsv, mosaic, mixup)
# Langsung skip ke Cell berikutnya (Training)

print("⚠️ SKIP MANUAL AUGMENTATION!")
print("   ✅ YOLO built-in augmentation sudah cukup untuk ~700-800 images")
print("   ✅ Langsung lanjut ke Cell Training (Step 5)")
print("\n💡 Built-in augmentation akan aktif saat training:")
print("   - HSV variations (brightness, saturation)")
print("   - Mosaic (combine 4 images)")
print("   - Mixup (blend images)")
print("   - Rotation, translation, scaling")

In [ ]:
# ⚠️ SKIP CELL INI JUGA!
# Function ini hanya untuk reference, JANGAN dijalankan

print("⚠️ CELL INI DI-SKIP!")
print("   Manual augmentation tidak efisien dan memakan waktu.")
print("   YOLO built-in augmentation lebih baik dan otomatis.")
print("\n🚀 Lanjut ke Step 5: Training!")

## 🏋️ Step 5: Train Model dengan Konfigurasi IMPROVED
### ✅ Improvements dari Original Training:
- ✅ **2 datasets merged** (2x lebih banyak data!)
- ✅ **YOLOv8s** (vs YOLOv8n - lebih akurat!)
- ✅ **150 epochs** (vs 100 - lebih banyak training time)
- ✅ **Patience 30** (vs 20 - early stopping lebih sabar)
- ✅ **Built-in augmentation** (HSV, mosaic, mixup - on-the-fly!)
- ✅ **AdamW optimizer** (vs SGD - better convergence)
- ✅ **ONNX export** (30-50% faster inference)

In [ ]:
from ultralytics import YOLO
import torch

# Check GPU availability
print(f"🖥️ CUDA Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"   GPU: {torch.cuda.get_device_name(0)}")
    device = 0
else:
    print("   Running on CPU (akan lebih lambat)")
    device = 'cpu'

# Load pretrained model
# Gunakan yolov8s.pt untuk balance antara speed dan accuracy
# Alternatif: yolov8n.pt (lebih cepat), yolov8m.pt (lebih akurat)
model = YOLO("yolov8s.pt")  # 's' = small (recommended)

print("\n✅ Model loaded successfully")

In [ ]:
# Train dengan konfigurasi IMPROVED
print("🔥 Starting training with IMPROVED configuration...")
print("="*60)

results = model.train(
    # Dataset (langsung dari Roboflow - sudah merged!)
    data=dataset_yaml_path,  # ✅ Merged dataset dari Roboflow v3
    
    # Training parameters
    epochs=150,           # ⬆️ Lebih lama dari original (100 → 150)
    patience=30,          # ⬆️ Early stopping lebih sabar (20 → 30)
    batch=16,             # Sesuaikan dengan VRAM GPU (16 recommended)
    imgsz=720,            # ✅ Real-world resolution
    
    # 🎨 Augmentation (built-in YOLO) - INI KUNCI UNTUK ROBUSTNESS!
    # Dataset TANPA augmentation → YOLO augmentation bekerja optimal!
    hsv_h=0.015,          # ✅ Hue variation (lighting changes)
    hsv_s=0.7,            # ✅ Saturation variation
    hsv_v=0.4,            # ✅ Brightness/Value variation
    degrees=5.0,          # ✅ Rotation (±5°) - slight angle changes
    translate=0.1,        # ✅ Translation (10% shift)
    scale=0.5,            # ✅ Scale variation (zoom in/out)
    shear=0.0,            # ❌ No shear (chess board tidak miring)
    perspective=0.0,      # ❌ No perspective (handled in preprocessing)
    flipud=0.0,           # ❌ No flip up-down (chess punya orientasi)
    fliplr=0.0,           # ❌ No flip left-right (a1 harus kiri bawah)
    mosaic=1.0,           # ✅✅ Mosaic augmentation (combine 4 images - POWERFUL!)
    mixup=0.1,            # ✅ Mixup augmentation (blend images)
    
    # 🎯 Optimization
    optimizer='AdamW',    # ✅ AdamW > SGD untuk dataset kecil-menengah
    lr0=0.001,            # Learning rate initial
    lrf=0.01,             # Learning rate final (1% dari lr0)
    momentum=0.937,       # Momentum untuk optimizer
    weight_decay=0.0005,  # Regularization
    warmup_epochs=3,      # ✅ Warmup 3 epochs untuk stabilitas
    warmup_momentum=0.8,
    warmup_bias_lr=0.1,
    
    # 🖥️ Hardware
    device=device,        # GPU or CPU
    workers=8,            # Data loading parallel workers
    
    # 💾 Saving & Logging
    project="chess_detection_improved",
    name="yolov8s_merged_v3",  # ✅ v3 = merged tanpa augmentasi
    exist_ok=True,
    pretrained=True,
    save=True,
    save_period=10,       # ✅ Save checkpoint tiap 10 epochs
    
    # 📊 Validation
    val=True,
    plots=True,           # ✅ Generate training plots
    
    # 🔧 Other
    verbose=True,
    seed=42,              # ✅ Reproducibility
)

print("\n" + "="*60)
print("✅ Training completed!")
print("="*60)
print(f"📦 Best model: {results.save_dir}/weights/best.pt")
print(f"📦 Last model: {results.save_dir}/weights/last.pt")
print(f"\n💡 Next: Validate model (Step 6) → Export ONNX (Step 7)")

## 📊 Step 6: Evaluate Model Performance

In [ ]:
# Validasi model
metrics = model.val()

print("\n📊 Model Performance Metrics:")
print(f"   mAP@50: {metrics.box.map50:.4f}")
print(f"   mAP@50-95: {metrics.box.map:.4f}")
print(f"   Precision: {metrics.box.mp:.4f}")
print(f"   Recall: {metrics.box.mr:.4f}")
print(f"\n   Per-class AP@50:")

# Print per-class metrics
class_names = data_config.get('names', [])  # ✅ Fixed: menggunakan data_config dari Cell 8
for i, ap in enumerate(metrics.box.ap50):
    if i < len(class_names):
        print(f"      {class_names[i]}: {ap:.4f}")

## 🚀 Step 7: Export Model ke ONNX (CRITICAL untuk Speed Improvement)
### ONNX = 30-50% lebih cepat dari PyTorch!

In [ ]:
# Export best model ke ONNX
best_model_path = f"{results.save_dir}/weights/best.pt"
best_model = YOLO(best_model_path)

# Export dengan optimasi
onnx_path = best_model.export(
    format='onnx',
    dynamic=False,        # Static shape untuk speed
    simplify=True,        # ✅ Simplify graph (lebih cepat)
    opset=12,             # ONNX opset version
)

print(f"\n✅ Model exported to ONNX: {onnx_path}")
print(f"\n📦 Files ready for deployment:")
print(f"   1. PyTorch model: {best_model_path}")
print(f"   2. ONNX model: {onnx_path}")
print(f"\n💡 Use ONNX model in production for 30-50% speed boost!")

## 🧪 Step 8: Test Inference Speed (PyTorch vs ONNX)

In [ ]:
import time
import numpy as np

# Dummy image untuk testing
dummy_image = np.random.randint(0, 255, (720, 720, 3), dtype=np.uint8)

# Test PyTorch model
print("⏱️ Testing PyTorch model speed...")
pytorch_times = []
for i in range(30):
    start = time.time()
    _ = best_model(dummy_image, verbose=False)
    pytorch_times.append(time.time() - start)

pytorch_avg = np.mean(pytorch_times[10:])  # Skip first 10 (warmup)
pytorch_fps = 1 / pytorch_avg

print(f"   PyTorch: {pytorch_avg*1000:.2f} ms/frame ({pytorch_fps:.1f} FPS)")

# Test ONNX model
print("\n⏱️ Testing ONNX model speed...")
try:
    import onnxruntime as ort
    
    # Load ONNX model
    onnx_session = ort.InferenceSession(onnx_path, providers=['CUDAExecutionProvider', 'CPUExecutionProvider'])
    
    onnx_times = []
    for i in range(30):
        # Prepare input (ONNX expects specific format)
        input_tensor = dummy_image.transpose(2, 0, 1).astype(np.float32) / 255.0
        input_tensor = np.expand_dims(input_tensor, axis=0)
        
        start = time.time()
        _ = onnx_session.run(None, {onnx_session.get_inputs()[0].name: input_tensor})
        onnx_times.append(time.time() - start)
    
    onnx_avg = np.mean(onnx_times[10:])
    onnx_fps = 1 / onnx_avg
    
    print(f"   ONNX: {onnx_avg*1000:.2f} ms/frame ({onnx_fps:.1f} FPS)")
    
    speedup = pytorch_avg / onnx_avg
    print(f"\n🚀 ONNX is {speedup:.2f}x faster than PyTorch!")
    
except Exception as e:
    print(f"   ⚠️ ONNX testing failed: {e}")
    print("   Install onnxruntime: pip install onnxruntime-gpu")

## 📈 Step 9: Visualize Training Results

In [ ]:
from IPython.display import Image, display
import os

# Show training plots
results_dir = results.save_dir

print("📊 Training Visualizations:\n")

plots = [
    'results.png',
    'confusion_matrix.png',
    'F1_curve.png',
    'P_curve.png',
    'R_curve.png',
    'PR_curve.png'
]

for plot in plots:
    plot_path = os.path.join(results_dir, plot)
    if os.path.exists(plot_path):
        print(f"\n{plot}:")
        display(Image(filename=plot_path, width=800))
    else:
        print(f"⚠️ {plot} not found")

## 📝 Step 10: Model Comparison & Documentation

In [ ]:
import json

# Create comprehensive model card
model_card = {
    "model_name": "Chess Detection YOLOv8s - Improved & Merged",
    "version": "v3.0",
    "base_model": "yolov8s.pt",
    
    "dataset": {
        "source": "Roboflow merged dataset (detection-chess v3)",
        "dataset_1": "kosan-hendra (~740 images ORIGINAL)",
        "dataset_2": "detection-chess FIXED (~137 images, 12 classes)",
        "total": "~877 images",
        "augmentation": "NONE in Roboflow (original images only!)",
        "num_classes": data_config.get('nc', 0),
        "classes": data_config.get('names', []),
    },
    
    "training": {
        "epochs": 150,
        "patience": 30,
        "image_size": 720,
        "batch_size": 16,
        "optimizer": "AdamW",
        "augmentation": "HSV, mosaic, mixup (built-in YOLO - on-the-fly!)",
    },
    
    "performance": {
        "map50": float(metrics.box.map50),
        "map50_95": float(metrics.box.map),
        "precision": float(metrics.box.mp),
        "recall": float(metrics.box.mr),
    },
    
    "improvements_vs_original": [
        "✅ Dataset merged di Roboflow (1 download saja!)",
        "✅ Dataset ORIGINAL (740 + 137, NO pre-augmentation!)",
        "✅ Dataset 2 FIXED (12 classes, bukan 6!)",
        "✅ YOLOv8s vs YOLOv8n (+250% params)",
        "✅ Built-in YOLO augmentation (on-the-fly dynamic!)",
        "✅ 150 epochs vs 100 (+50%)",
        "✅ Patience 30 vs 20 (+50%)",
        "✅ AdamW vs SGD optimizer",
        "✅ ONNX export (30-50% faster)",
    ],
    
    "files": {
        "pytorch_model": str(best_model_path),
        "onnx_model": str(onnx_path),
    },
}

# Save model card
model_card_path = os.path.join(results.save_dir, 'model_card.json')
with open(model_card_path, 'w') as f:
    json.dump(model_card, f, indent=2)

print("📄 MODEL CARD:")
print(json.dumps(model_card, indent=2))
print(f"\n✅ Saved to: {model_card_path}")

## 🎯 SUMMARY: Training Completed Successfully! 🎉

### ✅ Improvements vs Original:
1. **Dataset merged di Roboflow** - 1 project, 1 download (simpler workflow!)
2. **Dataset ORIGINAL** - 740 + 137 = 877 images (NO pre-augmentation!)
3. **Dataset 2 FIXED** - 12 classes (bukan 6 yang salah!)
4. **Model bigger** - YOLOv8s vs YOLOv8n (+250% params)
5. **Training longer** - 150 epochs vs 100 (+50%)
6. **Early stopping better** - Patience 30 vs 20
7. **YOLO augmentation optimal** - On-the-fly dynamic (tidak double augmentation!)
8. **Optimizer better** - AdamW vs SGD
9. **ONNX export** - 30-50% faster inference

### 📊 Expected Performance:
```
Original: mAP@50 ~0.87, False Positives HIGH ❌
Improved: mAP@50 ~0.93-0.97, False Positives LOW ✅
```

### 🎨 Augmentation Strategy:
```
Dataset: 877 original images (NO pre-augmentation)
+ YOLO built-in: HSV, mosaic, mixup (dynamic per epoch)
= ~8,770 variations per epoch!
= Better generalization vs static augmentation ✅
```

### 📦 Next Steps:
1. Copy models to Google Drive (next cell)
2. Download to: `d:/chess-detection-improve/app/model/`
3. Test app dengan model baru
4. Verify false positives berkurang!

---
**🎉 Training completed with OPTIMAL dataset (merged + original + 12 classes)! ♟️**

In [ ]:
import os
import shutil

print("💾 SAVING MODELS TO GOOGLE DRIVE\n")

# Copy ke Google Drive
drive_save_path = "/content/drive/MyDrive/chess_detection_models_improved"
os.makedirs(drive_save_path, exist_ok=True)

weights_dir = f"{results.save_dir}/weights"

# Copy best.pt
best_src = f"{weights_dir}/best.pt"
if os.path.exists(best_src):
    shutil.copy2(best_src, f"{drive_save_path}/best.pt")
    print(f"✅ best.pt saved to Drive")

# Copy best.onnx
onnx_src = f"{weights_dir}/best.onnx"
if os.path.exists(onnx_src):
    shutil.copy2(onnx_src, f"{drive_save_path}/best.onnx")
    print(f"✅ best.onnx saved to Drive")

# Copy results
if os.path.exists(f"{results.save_dir}/results.png"):
    shutil.copy2(f"{results.save_dir}/results.png", f"{drive_save_path}/training_results.png")
    print(f"✅ results.png saved to Drive")

print(f"\n🎉 Models saved to: {drive_save_path}")
print(f"\n📥 Download from: drive.google.com → MyDrive/chess_detection_models_improved/")
print(f"💡 Copy best.onnx to: d:/chess-detection-improve/app/model/")

## 📥 OPTIONAL: Download Directly to Computer

In [ ]:
# ALTERNATIVE: Download langsung dari Colab
from google.colab import files

print("📥 Downloading models to your computer...\n")

try:
    # Download best.pt
    files.download(f"{drive_save_path}/best.pt")
    print("✅ best.pt downloaded!")
    
    # Download best.onnx
    if os.path.exists(f"{drive_save_path}/best.onnx"):
        files.download(f"{drive_save_path}/best.onnx")
        print("✅ best.onnx downloaded!")
    
    # Download results
    if os.path.exists(f"{drive_save_path}/training_results.png"):
        files.download(f"{drive_save_path}/training_results.png")
        print("✅ training_results.png downloaded!")
    
    print("\n🎉 Check your Downloads folder!")
    print("\n📂 Copy to:")
    print("   best.onnx → d:/chess-detection-improve/app/model/best.onnx")
    
except Exception as e:
    print(f"❌ Error: {e}")
    print(f"\n💡 Alternative: Download from Google Drive manually")
    print(f"   Location: {drive_save_path}")

## 💾 CRITICAL: Save Model ke Google Drive!
### ⚠️ Model di `/content/` akan HILANG saat runtime disconnect!